<a href="https://colab.research.google.com/github/Shi-Yile/SPH6004-Assignment-1/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Import packages

# Data-processing packages
import pandas as pd
import numpy as np

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sn

# ML packages
import sklearn

import os
from google.colab import drive

In [ ]:
# give permission to load data from google drive
drive.mount('/content/drive')
# os.chdir('drive/My Drive/SPH6004/Assignment-1')

In [37]:
# load dataset
df = pd.read_csv('data.csv')
df_origin = df.copy()
df.head()

,id,aki,gender,admission_age,race,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,...,ggt_max,ld_ldh_min,ld_ldh_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,height,weight_admit
0,36570066,3,F,79.953141,BLACK/AFRICAN AMERICAN,96.0,104.0,100.083333,103.0,126.0,...,NaN,236.0,318.0,15.0,6.0,5.0,4.0,0.0,157.0,110.0
1,39307659,0,F,78.194169,WHITE - RUSSIAN,72.0,134.0,97.263158,97.0,127.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,82.0
2,38743306,2,F,65.602396,WHITE,60.0,97.0,84.166667,95.0,143.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,62.1
3,32339865,2,F,64.906629,UNKNOWN,59.0,87.0,71.461538,113.0,150.0,...,NaN,NaN,NaN,15.0,1.0,0.0,1.0,1.0,170.0,113.1
4,35526987,2,M,57.438861,WHITE,57.0,100.0,82.387097,81.0,127.0,...,NaN,NaN,NaN,15.0,NaN,0.0,1.0,1.0,178.0,97.4


In [38]:
# basic infomration of each column in df
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50920 entries, 0 to 50919
Data columns (total 162 columns):
 #    Column                  Non-Null Count  Dtype  
---   ------                  --------------  -----  
 0    id                      50920 non-null  int64  
 1    aki                     50920 non-null  int64  
 2    gender                  50920 non-null  object 
 3    admission_age           50920 non-null  float64
 4    race                    50920 non-null  object 
 5    heart_rate_min          50841 non-null  float64
 6    heart_rate_max          50841 non-null  float64
 7    heart_rate_mean         50841 non-null  float64
 8    sbp_min                 50823 non-null  float64
 9    sbp_max                 50823 non-null  float64
 10   sbp_mean                50823 non-null  float64
 11   dbp_min                 50823 non-null  float64
 12   dbp_max                 50823 non-null  float64
 13   dbp_mean                50823 non-null  float64
 14   mbp_min             

<ipython-input-38-1fc94d2bc7dc>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose = True, null_counts = True)


In [59]:
# calculate the proportion of NAs in each column
df_NA_prop = 1 - df.count() / len(df)
idx_NA10 = df_NA_prop[df_NA_prop <= 0.1].index
idx_NA10

Index(['id', 'aki', 'gender', 'admission_age', 'race', 'heart_rate_min',
       'heart_rate_max', 'heart_rate_mean', 'sbp_min', 'sbp_max', 'sbp_mean',
       'dbp_min', 'dbp_max', 'dbp_mean', 'mbp_min', 'mbp_max', 'mbp_mean',
       'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'temperature_min',
       'temperature_max', 'temperature_mean', 'spo2_min', 'spo2_max',
       'spo2_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'hematocrit_min.1', 'hematocrit_max.1', 'hemoglobin_min.1',
       'hemoglobin_max.1', 'platelets_min', 'platelets_max', 'wbc_min',
       'wbc_max', 'aniongap_min', 'aniongap_max', 'bicarbonate_min.1',
       'bicarbonate_max.1', 'bun_min', 'bun_max', 'calcium_min.1',
       'calcium_max.1', 'chloride_min.1', 'chloride_max.1', 'glucose_min.2',
       'glucose_max.2', 'sodium_min.1', 'sodium_max.1', 'potassium_min.1',
       'potassium_max.1', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'gcs_min',
       'gcs_motor', 'gcs_verbal', 'gcs_eyes', 'gcs_u

In [55]:
# extract a subset of df with columns including NAs less than 10%
df_sub_NA10 = df[idx_NA10]
df_sub_NA10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50920 entries, 0 to 50919
Data columns (total 63 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 50920 non-null  int64  
 1   aki                50920 non-null  int64  
 2   gender             50920 non-null  object 
 3   admission_age      50920 non-null  float64
 4   race               50920 non-null  object 
 5   heart_rate_min     50841 non-null  float64
 6   heart_rate_max     50841 non-null  float64
 7   heart_rate_mean    50841 non-null  float64
 8   sbp_min            50823 non-null  float64
 9   sbp_max            50823 non-null  float64
 10  sbp_mean           50823 non-null  float64
 11  dbp_min            50823 non-null  float64
 12  dbp_max            50823 non-null  float64
 13  dbp_mean           50823 non-null  float64
 14  mbp_min            50805 non-null  float64
 15  mbp_max            50805 non-null  float64
 16  mbp_mean           508

In [64]:
df_sub_com10['race'].value_counts()

WHITE                                        24528
UNKNOWN                                       4525
BLACK/AFRICAN AMERICAN                        2874
OTHER                                         1345
WHITE - OTHER EUROPEAN                         702
UNABLE TO OBTAIN                               515
ASIAN                                          472
HISPANIC/LATINO - PUERTO RICAN                 409
ASIAN - CHINESE                                406
HISPANIC OR LATINO                             362
WHITE - RUSSIAN                                330
HISPANIC/LATINO - DOMINICAN                    253
PATIENT DECLINED TO ANSWER                     252
BLACK/CAPE VERDEAN                             239
BLACK/CARIBBEAN ISLAND                         216
BLACK/AFRICAN                                  154
ASIAN - SOUTH EAST ASIAN                       136
PORTUGUESE                                     123
WHITE - EASTERN EUROPEAN                       101
ASIAN - ASIAN INDIAN           

In [66]:
df_sub_com10['gender'].value_counts()

M    21509
F    17122
Name: gender, dtype: int64

In [68]:
# recode gender as 0-1  0: male, 1: female
df_sub_com10['gender_'] = df_sub_com10['gender'].apply(lambda x:int(x == 'F'))
df_sub_com10['gender_'].value_counts()

<ipython-input-68-bbe89ddf892b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_com10['gender_'] = df_sub_com10['gender'].apply(lambda x:int(x == 'F'))


0    21509
1    17122
Name: gender_, dtype: int64

In [70]:
df_sub_com10['aki'].value_counts()

2    12570
0    12147
1     7368
3     6546
Name: aki, dtype: int64

In [72]:
# re-categorise AKI as a binary column  0: aki = 0, 1: aki = 1/2/3
df_sub_com10['aki_binary'] = df_sub_com10['aki'].apply(lambda x:int(x == 1 or x == 2 or x == 3))
df_sub_com10['aki_binary'].value_counts()

<ipython-input-72-fcd846134705>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_com10['aki_binary'] = df_sub_com10['aki'].apply(lambda x:int(x == 1 or x == 2 or x == 3))


1    26484
0    12147
Name: aki_binary, dtype: int64

In [ ]:
# create outcome vector and predictor matrix
